<a href="https://colab.research.google.com/github/SebastianLarssonDTU/02456-Reinforcement-Learning-Project/blob/before_framestacking/Auto_Combining_from_multiple_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INIT : Procgen, Drive, Git

In [1]:
!pip install procgen

     |████████████████████████████████| 39.9MB 105kB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 204kB 46.0MB/s 
     |████████████████████████████████| 665kB 34.4MB/s 
     |████████████████████████████████| 3.3MB 40.4MB/s 
     |████████████████████████████████| 22.2MB 1.6MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: imageio 2.4.1
    Uninstalling imageio-2.4.1:
      Successfully uninstalled imageio-2.4.1


In [2]:
#Clone git
!git clone -b restructure_code https://github.com/SebastianLarssonDTU/02456-Reinforcement-Learning-Project.git "my_project"

Cloning into 'my_project'...
remote: Enumerating objects: 146, done.
remote: Counting objects: 100% (146/146), done.
remote: Compressing objects: 100% (146/146), done.
remote: Total 606 (delta 88), reused 2 (delta 0), pack-reused 460
Receiving objects: 100% (606/606), 9.10 MiB | 21.36 MiB/s, done.
Resolving deltas: 100% (371/371), done.


In [3]:
#update git
%cd /content/my_project
! git pull

/content/my_project
Already up to date.


In [4]:
import datatools as tools
from datatools import DATA_PATH, MODEL_PATH
#Mount drive
tools.mount_drive()

Mounted at /content/drive


# Import

In [5]:
#Import all custom files
import baseline
import datatools as tools
import hyperparameters as h
import model
import my_util
import policy
import ppo
import utils

#other imports
import torch

#import specific methods
from baseline import set_hyperparameters
from ppo import PPO
from experiments import run_experiment, print_list_of_experiments


# Create Index File



In [6]:
INDEX = tools.create_index_table_from_txt_files()

# Create .csv's

Assumes that all file_names are correct! (with step count etc)

## Code

In [9]:
import pandas as pd

def create_training_csv_from_multiple_runs(base_file_name):
  all_related_files = find_related_csv_files(base_file_name)

  #create a df for each file
  dfs = []
  for name in all_related_files:
    dfs.append(pd.read_csv(DATA_PATH + name + ".csv"))

  dfs = remove_overlap_from_dfs(dfs)

  #Concatenate
  final_df = pd.concat(dfs)
  final_df = final_df.set_index("Step")

  #Save to file
  final_df.to_csv(DATA_PATH +base_file_name +"_combined.csv")

def create_EVAL_csv_from_multiple_runs(base_file_name):
  all_related_files = find_related_csv_files(base_file_name)
  base = 32*256*16

  dfs = []
  for name in all_related_files:
    df = pd.read_csv(DATA_PATH + name + "_EVAL.csv")
    #add steps if missing
    if "step" not in df.columns:
      df.insert(0, "Step", [base*(i+1) for i in range(len(df))])
    else:
      df.rename(columns={'step':'Step'}, inplace=True)
    dfs.append(df)

  dfs = remove_overlap_from_dfs(dfs)

  #Remove whitespace in headers
  for df in dfs:
    df.rename(columns=lambda x: x.strip(), inplace=True)

    #Concatenate
  final_df = pd.concat(dfs)
  final_df = final_df.set_index("Step")

  #save in file
  final_df.to_csv(DATA_PATH +base_file_name +"_combined_EVAL.csv")


def remove_overlap_from_dfs(dfs):
  #Dataframes overlap, so cut off excess
  breakpoints=[]
  for df in dfs[1:]:
    breakpoints.append(df['Step'][0])
  for i in range(len(dfs)-1):
    dfs[i] = dfs[i][dfs[i]["Step"] < breakpoints[i]]
  return dfs

def find_related_csv_files(base_file_name):
  file_names = [x.strip() for x in INDEX['file_name']]
  all_related_files= []
  for name in file_names:
    if base_file_name in name and "combined" not in name:
      all_related_files.append(name)

  return all_related_files


## Create files

In [10]:
base_file_name = "Experiment8_200levels_Run_06Dec_16h17m59s"
create_EVAL_csv_from_multiple_runs(base_file_name)
create_training_csv_from_multiple_runs(base_file_name)

In [11]:
base_file_name = "Experiment7_200levels_Run_06Dec_16h18m31s"
create_EVAL_csv_from_multiple_runs(base_file_name)
create_training_csv_from_multiple_runs(base_file_name)

In [12]:
base_file_name = "Experiment2_200levels_Run_07Dec_13h49m13s"
create_EVAL_csv_from_multiple_runs(base_file_name)
create_training_csv_from_multiple_runs(base_file_name)